# Binax Now Malaria Test Training Data Image Generation
### We need to generate training data for the following scenarios: 

#### Control Line Valid
- no malaria detected
- tl1 malaria detected
- tl2 malaria detected
- tl1 and tl2 malaria detected

#### Control Line Invalid*
- no malaria detected
- tl1 malaria detected
- tl2 malaria detected
- tl1 and tl2 malaria detected

##### *Not sure if the control line being invalid affects the results of any of the other detection results

#### TRAINING DATA LABELS
- valid, no malaria detected (0)
- valid, tl1 only detected (1)
- valid, tl2 only detected (2)
- valid, tl1 and tl2 malaria detected (3)
- invalid (4)


In [156]:
from PIL import Image, ImageDraw, ImageFont, ImageFilter
import random

### THIS SHOULD BE ALL OF THE REQUIRED DATA NEEDED IN ORDER TO DYNAMICALLY GENERATE TRAINING DATA ###
BINAX_NOW_PACKAGE_PIL_IMAGE = Image.open('BinaxPackage.png')  
BINAX_PACKAGE_IMAGE_WIDTH, BINAX_PACKAGE_IMAGE_HEIGHT = pil_image.size # in pixels 
BINAX_PACKAGE_WIDTH = 64 # mm
BINAX_PACKAGE_HEIGHT = 76 # mm
BINAX_PACKAGE_DIST_TO_STRIP_FROM_Y = 24. # mm (first guess was 28 mm)
BINAX_PACKAGE_DIST_TO_STRIP_FROM_X = 26.5 # mm (first guess was 28 mm)
BINAX_WIDTH = 6 # mm
BINAX_HEIGHT = 14. # mm
LINE_HEIGHT = 0.6 # mm
CL_HEIGHT_POSITION = 5 # relative to top, in mm
TL1_HEIGHT_POSITION = 7 # relative to top
TL2_HEIGHT_POSITION = 9 # relative to top
#DISTANCE_BETWEEN_LINES = 2. # mm
### END REQUIRED DATA NEEDED TO DYNAMICALLY GENERATE TRAINING DATA

IMAGE_WIDTH = (BINAX_WIDTH * 1.0 / BINAX_PACKAGE_WIDTH) * BINAX_PACKAGE_IMAGE_WIDTH # in px
HEIGHT_TO_WIDTH_RATIO = BINAX_HEIGHT / BINAX_WIDTH
IMAGE_HEIGHT = HEIGHT_TO_WIDTH_RATIO * IMAGE_WIDTH

STRIP_TOP_LEFT_POS_X = (BINAX_PACKAGE_DIST_TO_STRIP_FROM_X / BINAX_PACKAGE_WIDTH) * BINAX_PACKAGE_IMAGE_WIDTH
STRIP_TOP_LEFT_POS_Y = (BINAX_PACKAGE_DIST_TO_STRIP_FROM_Y / BINAX_PACKAGE_HEIGHT) * BINAX_PACKAGE_IMAGE_HEIGHT

CROP_TOP_LEFT_X = 21 # mm
CROP_TOP_LEFT_Y = 20 # mm
BOTTOM_RIGHT_X = 36 # mm
BOTTOM_RIGHT_Y = 42 # mm

CROP_TOP_LEFT_IMAGE_POS_X = (CROP_TOP_LEFT_X / BINAX_PACKAGE_WIDTH) * BINAX_PACKAGE_IMAGE_WIDTH
CROP_TOP_LEFT_IMAGE_POS_Y = (CROP_TOP_LEFT_Y / BINAX_PACKAGE_HEIGHT) * BINAX_PACKAGE_IMAGE_HEIGHT
CROP_BOTTOM_RIGHT_IMAGE_POS_X = (BOTTOM_RIGHT_X / BINAX_PACKAGE_WIDTH) * BINAX_PACKAGE_IMAGE_WIDTH
CROP_BOTTOM_RIGHT_IMAGE_POS_Y = (BOTTOM_RIGHT_Y / BINAX_PACKAGE_HEIGHT) * BINAX_PACKAGE_IMAGE_HEIGHT

CL_POSITION_TO_HEIGHT_RATIO = CL_HEIGHT_POSITION / BINAX_HEIGHT
TL1_POSITION_TO_HEIGHT_RATIO = TL1_HEIGHT_POSITION / BINAX_HEIGHT
TL2_POSITION_TO_HEIGHT_RATIO = TL2_HEIGHT_POSITION / BINAX_HEIGHT

CL_POSITION_MIDDLE = CL_POSITION_TO_HEIGHT_RATIO * IMAGE_HEIGHT
CL_POSITION_TOP = CL_POSITION_MIDDLE - ((LINE_HEIGHT / 2) / BINAX_HEIGHT) * IMAGE_HEIGHT
CL_POSITION_BOTTOM = CL_POSITION_TOP + ((LINE_HEIGHT / 2) / BINAX_HEIGHT) * IMAGE_HEIGHT
CL_POSITION_COORDINATES = (0, CL_POSITION_TOP, IMAGE_WIDTH, CL_POSITION_BOTTOM)

TL1_POSITION_MIDDLE = TL1_POSITION_TO_HEIGHT_RATIO * IMAGE_HEIGHT
TL1_POSITION_TOP = TL1_POSITION_MIDDLE - ((LINE_HEIGHT / 2) / BINAX_HEIGHT) * IMAGE_HEIGHT
TL1_POSITION_BOTTOM = TL1_POSITION_TOP + ((LINE_HEIGHT / 2) / BINAX_HEIGHT) * IMAGE_HEIGHT
TL1_POSITION_COORDINATES = (0, TL1_POSITION_TOP, IMAGE_WIDTH, TL1_POSITION_BOTTOM)

TL2_POSITION_MIDDLE = TL2_POSITION_TO_HEIGHT_RATIO * IMAGE_HEIGHT
TL2_POSITION_TOP = TL2_POSITION_MIDDLE - ((LINE_HEIGHT / 2) / BINAX_HEIGHT) * IMAGE_HEIGHT
TL2_POSITION_BOTTOM = TL2_POSITION_TOP + ((LINE_HEIGHT / 2) / BINAX_HEIGHT) * IMAGE_HEIGHT
TL2_POSITION_COORDINATES = (0, TL2_POSITION_TOP, IMAGE_WIDTH, TL2_POSITION_BOTTOM)

STRIP_DIMENSIONS=(int(IMAGE_WIDTH), int(IMAGE_HEIGHT))
CL_POSITION = CL_POSITION_COORDINATES
TL1_POSITION = TL1_POSITION_COORDINATES
TL2_POSITION = TL2_POSITION_COORDINATES

DEFAULT_BACKGROUND_COLOR = (250, 250, 250) # I picked 230, 230, 230 instead of 255, 255, 255 to better represent background color of strip in real life settings
DEFAULT_CONTROL_COLOR=(255, 0, 0)
DEFAULT_TL1_COLOR=(255, 0, 0)
DEFAULT_TL2_COLOR=(255, 0, 10)

In [157]:
def draw_binax(
        control_color=DEFAULT_CONTROL_COLOR, 
        tl_color=DEFAULT_TL1_COLOR, 
        t2_color=DEFAULT_TL2_COLOR, 
        output_name='pil_text.png',
        plus_minus_offset_value=1,
        pixel_offset_value=0):  
    strip = Image.new('RGB', STRIP_DIMENSIONS, color = DEFAULT_BACKGROUND_COLOR)

    draw_rectangle(strip, CL_POSITION, control_color)
    draw_rectangle(strip, TL1_POSITION, tl_color)
    draw_rectangle(strip, TL2_POSITION, t2_color)
    final_strip = strip.filter(ImageFilter.GaussianBlur(radius = 2))
    BINAX_NOW_PACKAGE_PIL_IMAGE.paste(final_strip, (int(STRIP_TOP_LEFT_POS_X), int(STRIP_TOP_LEFT_POS_Y)))
    BINAX_NOW_PACKAGE_PIL_IMAGE.save(output_name) # this is the full package with the strip on it
    
    ### CROPPING SECTION OF THIS METHOD ###
    # now let's crop this to be just the portion we want with these defaults
    top_left_x = CROP_TOP_LEFT_IMAGE_POS_X + plus_minus_offset_value*pixel_offset_value
    top_left_y = CROP_TOP_LEFT_IMAGE_POS_Y + plus_minus_offset_value*pixel_offset_value
    bottom_right_x = CROP_BOTTOM_RIGHT_IMAGE_POS_X + plus_minus_offset_value*pixel_offset_value
    bottom_right_y = CROP_BOTTOM_RIGHT_IMAGE_POS_Y + plus_minus_offset_value*pixel_offset_value
    
    full_image = Image.open(output_name)
    cropped_image = full_image.crop((top_left_x, top_left_y, bottom_right_x, bottom_right_y)) 
    cropped_image.save(output_name)

def draw_rectangle(pil_image, position, color):
    line = ImageDraw.Draw(pil_image) 
    line.rectangle(position, fill=color)

### LABEL SHOULD BE 0 ###
def generate_valid_test_no_malaria(sample_count = 1000, test_data = False):
    
    # the valid color range was decided based on manual inspection
    valid_color_range = [(i, i, i) for i in range(250)]    
    invalid_color_range = [(i, i, i) for i in range(251, 256)]
    
    for i in range(sample_count):
        # sample the valid color range array for a color for control lines and the test lines
        cl_color = random.choice(valid_color_range)
        tl1_color = random.choice(invalid_color_range)
        tl2_color = random.choice(invalid_color_range)

        # generate a filename
        cl_str = text = '-'.join(str(x) for x in cl_color)
        tl1_str = text = '-'.join(str(x) for x in tl1_color)
        tl2_str = text = '-'.join(str(x) for x in tl2_color)
        if test_data:
            filename = 'test/0/' + str(i) + '_0.png'
        else:
            filename = 'train/0/' + str(i) + '_0.png'
        
        # call draw_binax to generate the actual image
        draw_binax(cl_color, tl1_color, tl2_color, filename)

### LABEL SHOULD BE A 1###
def generate_valid_test_t1_positive_only(sample_count = 1000, test_data = False):
    
    # the valid and invalid color ranges were decided based on manual inspection
    valid_color_range = [(i, i, i) for i in range(251)]    
    invalid_color_range = [(i, i, i) for i in range(251, 256)]
    
    for i in range(sample_count):
        # sample the valid color range array for a color for control lines and the test lines
        cl_color = random.choice(valid_color_range)
        tl1_color = random.choice(valid_color_range)
        tl2_color = random.choice(invalid_color_range)

        # generate a filename
        cl_str = text = '-'.join(str(x) for x in cl_color)
        tl1_str = text = '-'.join(str(x) for x in tl1_color)
        tl2_str = text = '-'.join(str(x) for x in tl2_color)
        if test_data:
            filename = 'test/1/' + str(i) + '_1.png'
        else:
            filename = 'train/1/' + str(i) + '_1.png'
        
        # call draw_binax to generate the actual image
        draw_binax(cl_color, tl1_color, tl2_color, filename)

### LABEL SHOULD BE A 2###
def generate_valid_test_t2_positive_only(sample_count = 1000, test_data = False):
    
    # the valid and invalid color ranges were decided based on manual inspection
    valid_color_range = [(i, i, i) for i in range(250)]    
    invalid_color_range = [(i, i, i) for i in range(251, 256)]
    
    for i in range(sample_count):
        # sample the valid color range array for a color for control lines and the test lines
        cl_color = random.choice(valid_color_range)
        tl1_color = random.choice(invalid_color_range)
        tl2_color = random.choice(valid_color_range)

        # generate a filename
        cl_str = text = '-'.join(str(x) for x in cl_color)
        tl1_str = text = '-'.join(str(x) for x in tl1_color)
        tl2_str = text = '-'.join(str(x) for x in tl2_color)
        if test_data:
            filename = 'test/2/' + str(i) + '_2.png'
        else:
            filename = 'train/2/' + str(i) + '_2.png'
        
        # call draw_binax to generate the actual image
        draw_binax(cl_color, tl1_color, tl2_color, filename)


### LABEL SHOULD BE A 3###
def generate_valid_test_positive_for_both_malarias(sample_count = 1000, test_data = False):
    
    # the valid color range was decided based on manual inspection
    valid_color_range = [(i, i, i) for i in range(250)]    
    
    for i in range(sample_count):
        # sample the valid color range array for a color for control lines and the test lines
        cl_color = random.choice(valid_color_range)
        tl1_color = random.choice(valid_color_range)
        tl2_color = random.choice(valid_color_range)

        # generate a filename
        cl_str = text = '-'.join(str(x) for x in cl_color)
        tl1_str = text = '-'.join(str(x) for x in tl1_color)
        tl2_str = text = '-'.join(str(x) for x in tl2_color)
        if test_data:
            filename = 'test/3/' + str(i) + '_3.png'
        else:
            filename = 'train/3/' + str(i) + '_3.png'
        
        # call draw_binax to generate the actual image
        draw_binax(cl_color, tl1_color, tl2_color, filename)


### LABEL SHOULD BE 4 ###
def generate_invalid_randomize_positive(sample_count = 1000, test_data = False):
    # the valid and invalid color ranges were decided based on manual inspection
    full_color_range = [(i, i, i) for i in range(256)]    
    invalid_color_range = [(i, i, i) for i in range(0, 256)]
    
    for i in range(sample_count):
        # sample the valid color range array for a color for control lines and the test lines
        cl_color = random.choice(invalid_color_range)
        tl1_color = random.choice(full_color_range)
        tl2_color = random.choice(full_color_range)

        # generate a filename
        if test_data:
            filename = 'test/4/' + str(i) + '_4.png'
        else:
            filename = 'train/4/' + str(i) + '_4.png'
        
        # call draw_binax to generate the actual image
        draw_binax(cl_color, tl1_color, tl2_color, filename)
    



In [ ]:
# pil_image = Image.open('BinaxNowPackage.png')  

# line = ImageDraw.Draw(pil_image) 
# line.rectangle(((200, 200), (300, 300)), fill="red")
  
# pil_image.save('BinaxNowPackage.png')


In [ ]:
# def draw_binax(control_color=DEFAULT_CONTROL_COLOR, tl_color=DEFAULT_TL1_COLOR, t2_color=DEFAULT_TL2_COLOR, output_name='pil_text.png'):  
#     strip = Image.new('RGB', STRIP_DIMENSIONS, color = DEFAULT_BACKGROUND_COLOR)
    
#     cl = ImageDraw.Draw(strip) 
#     cl.rectangle(CL_POSITION, fill=control_color)
    
#     tl1 = ImageDraw.Draw(strip) 
#     tl1.rectangle(TL1_POSITION, fill=tl_color)

#     tl2 = ImageDraw.Draw(strip) 
#     tl2.rectangle(TL2_POSITION, fill=t2_color)

#     strip.save(output_name)


In [ ]:
# add blur to rectangle
#     RADIUS = 10
#     diam = 2*RADIUS
#     back = Image.new('RGB', (pil_image.size[0]+diam, pil_image.size[1]+diam), (255,255,255))
#     back.paste(pil_image, (RADIUS, RADIUS))

#     mask = Image.new('L', (pil_image.size[0]+diam, pil_image.size[1]+diam), 255)
#     blck = Image.new('L', (pil_image.size[0]-diam, pil_image.size[1]-diam), 0)
#     mask.paste(blck, (diam, diam)) 

#     # Blur image and paste blurred edge according to mask
#     blur = back.filter(ImageFilter.GaussianBlur(RADIUS/2))
#     back.paste(blur, mask=mask)



In [ ]:
draw_binax()


In [ ]:
# fontsize = 5
# font = ImageFont.truetype("Arial.ttf", fontsize)
# height = 10

# new_range  = [(255, i, i) for i in range(255)]

# source_img = Image.new('RGB', (100, 2550), color = DEFAULT_BACKGROUND_COLOR)

# draw = ImageDraw.Draw(source_img)

# for i, val in enumerate(new_range):
#     start_point = (0, i*height)
#     middle_point = (start_point[0] + height/2, start_point[1] + height/2)
#     end_point = (100, i*height + height)
#     draw.rectangle((start_point, end_point), fill=val)
#     text = '-'.join(str(x) for x in val)
#     draw.text(middle_point, text, font=font)

# source_img.save('test.png')


In [ ]:
# fontsize = 10
# font = ImageFont.truetype("Arial.ttf", fontsize)
# height = 10

# new_range  = [(255, i, i) for i in range(255)]


# for i, val in enumerate(new_range):
#     source_img = Image.new('RGB', (100, 40), color = DEFAULT_BACKGROUND_COLOR)
#     draw = ImageDraw.Draw(source_img)
#     draw.rectangle(((0, 0), (100, 20)), fill=val)
#     text = '-'.join(str(x) for x in val)
#     draw.text((0, 0), text, font=font)
#     source_img.save(text + '.png')


In [ ]:
# def paste_binax_stripe_onto_binax_package(binax_package_image, binax_strip_image):
#     binax_package_image.paste(binax_strip_image, (int(STRIP_TOP_LEFT_POS_X), int(STRIP_TOP_LEFT_POS_Y)))
#     binax_package_image.save('binax_output.png')


In [ ]:
# def generate_valid_test_positive_for_both_malarias():
#     # we'll go to 250 instead of 255 because up to 250 is what we think should be counted as a recognizable color
#     cl_colors = [(i, i, i) for i in range(250)]
#     tl1_colors = cl_colors
#     tl2_colors = cl_colors
    
    
#     for i, cl in enumerate(cl_colors):
#         for j, tl1 in enumerate(tl1_colors):
#             for k, tl2 in enumerate(tl2_colors):
#                 cl_str = text = '-'.join(str(x) for x in cl)
#                 tl1_str = text = '-'.join(str(x) for x in tl1)
#                 tl2_str = text = '-'.join(str(x) for x in tl2)
#                 print(tl1)
#                 filename = 'valid_and_positive'+cl_str+'--'+tl1_str+'--'+tl2_str+'.png'
#                 draw_binax(cl,cl, tl2,filename)

# def generate_invalid_both_positive(sample_count = 1000):
#     # the valid and invalid color ranges were decided based on manual inspection
#     valid_color_range = [(i, i, i) for i in range(250)]    
#     invalid_color_range = [(i, i, i) for i in range(251, 256)]
    
#     for i in range(sample_count):
#         # sample the valid color range array for a color for control lines and the test lines
#         cl_color = random.choice(invalid_color_range)
#         tl1_color = random.choice(valid_color_range)
#         tl2_color = random.choice(valid_color_range)

#         # generate a filename
#         cl_str = text = '-'.join(str(x) for x in cl_color)
#         tl1_str = text = '-'.join(str(x) for x in tl1_color)
#         tl2_str = text = '-'.join(str(x) for x in tl2_color)
#         filename = 'invalid_and_both_positive'+cl_str+'--'+tl1_str+'--'+tl2_str+'.png'
        
#         # call draw_binax to generate the actual image
#         draw_binax(cl_color, tl1_color, tl2_color, filename)


# def generate_invalid_tl1_positive(sample_count = 1000):
#     # the valid and invalid color ranges were decided based on manual inspection
#     valid_color_range = [(i, i, i) for i in range(250)]    
#     invalid_color_range = [(i, i, i) for i in range(251, 256)]
    
#     for i in range(sample_count):
#         # sample the valid color range array for a color for control lines and the test lines
#         cl_color = random.choice(invalid_color_range)
#         tl1_color = random.choice(valid_color_range)
#         tl2_color = random.choice(invalid_color_range)

#         # generate a filename
#         cl_str = text = '-'.join(str(x) for x in cl_color)
#         tl1_str = text = '-'.join(str(x) for x in tl1_color)
#         tl2_str = text = '-'.join(str(x) for x in tl2_color)
#         filename = 'invalid_and_tl1_positive'+cl_str+'--'+tl1_str+'--'+tl2_str+'.png'
        
#         # call draw_binax to generate the actual image
#         draw_binax(cl_color, tl1_color, tl2_color, filename)

# def generate_invalid_tl2_position(sample_count = 1000):
#     # the valid and invalid color ranges were decided based on manual inspection
#     valid_color_range = [(i, i, i) for i in range(250)]    
#     invalid_color_range = [(i, i, i) for i in range(251, 256)]
    
#     for i in range(sample_count):
#         # sample the valid color range array for a color for control lines and the test lines
#         cl_color = random.choice(invalid_color_range)
#         tl1_color = random.choice(invalid_color_range)
#         tl2_color = random.choice(valid_color_range)

#         # generate a filename
#         cl_str = text = '-'.join(str(x) for x in cl_color)
#         tl1_str = text = '-'.join(str(x) for x in tl1_color)
#         tl2_str = text = '-'.join(str(x) for x in tl2_color)
#         filename = 'invalid_and_tl2_positive'+cl_str+'--'+tl1_str+'--'+tl2_str+'.png'
        
#         # call draw_binax to generate the actual image
#         draw_binax(cl_color, tl1_color, tl2_color, filename)

# def generate_invalid_no_malaria(sample_count = 1000):
#     # the valid and invalid color ranges were decided based on manual inspection
#     valid_color_range = [(i, i, i) for i in range(250)]    
#     invalid_color_range = [(i, i, i) for i in range(251, 256)]
    
#     for i in range(sample_count):
#         # sample the valid color range array for a color for control lines and the test lines
#         cl_color = random.choice(invalid_color_range)
#         tl1_color = random.choice(invalid_color_range)
#         tl2_color = random.choice(invalid_color_range)

#         # generate a filename
#         cl_str = text = '-'.join(str(x) for x in cl_color)
#         tl1_str = text = '-'.join(str(x) for x in tl1_color)
#         tl2_str = text = '-'.join(str(x) for x in tl2_color)
#         filename = 'invalid_and_no_positive'+cl_str+'--'+tl1_str+'--'+tl2_str+'.png'
        
#         # call draw_binax to generate the actual image
#         draw_binax(cl_color, tl1_color, tl2_color, filename)
    


In [158]:
generate_valid_test_no_malaria(sample_count = 4000)
generate_valid_test_t1_positive_only(sample_count = 4000)
generate_valid_test_t2_positive_only(sample_count = 4000)
generate_valid_test_positive_for_both_malarias(sample_count = 4000)
generate_invalid_randomize_positive(sample_count = 4000)

In [159]:
### move the first 300 files from the train folder to the test folder and 300 from the train folder to the 
import os
import shutil

### 300 to test folder
for i in range(300):
    shutil.move("train/0/" + str(i) + "_0.png", "test/0/" + str(i) + "_0.png")
    shutil.move("train/1/" + str(i) + "_1.png", "test/1/" + str(i) + "_1.png")
    shutil.move("train/2/" + str(i) + "_2.png", "test/2/" + str(i) + "_2.png")
    shutil.move("train/3/" + str(i) + "_3.png", "test/3/" + str(i) + "_3.png")
    shutil.move("train/4/" + str(i) + "_4.png", "test/4/" + str(i) + "_4.png")

### 300 to validate folder
for i in range(301,601):
    shutil.move("train/0/" + str(i) + "_0.png", "validate/0/" + str(i) + "_0.png")
    shutil.move("train/1/" + str(i) + "_1.png", "validate/1/" + str(i) + "_1.png")
    shutil.move("train/2/" + str(i) + "_2.png", "validate/2/" + str(i) + "_2.png")
    shutil.move("train/3/" + str(i) + "_3.png", "validate/3/" + str(i) + "_3.png")
    shutil.move("train/4/" + str(i) + "_4.png", "validate/4/" + str(i) + "_4.png")
    

FileNotFoundError: [Errno 2] No such file or directory: 'validate/0/301_0.png'